- PSO optimization for continuous designs using the $F'WF$, where $W$ is a diagonal weights matrix
- Include process variables?

# Overview

This notebook will explore the MCMC hit-and-run approach for generating random samples inside a potentiall high-dimensional convex polytope defined by a set of linear constraints. The general algorithm is as follows:

1. Generate a random point inside the feasible region (this is NP-hard, probably using rejection sampling is the only option)
2. Choose a random direction inside the space by sampling uniformly over the surface of an `n`-dimensional hypersphere
3. Project a line from the sampled point to the intersection with the boundary of the polytope in the direction chosen in step 2
4. Sample from a uniform distribution along this line. This is the next sample
5. Repeat using the newly sampled point
6. After a burn period mixing will occur and the sampled points will approximate the target distribution

In [3]:
using Polyhedra
using CDDLib
using LinearAlgebra

# Represent Polytopes Using Linear Inequalities
First we need to define the constraints for an H-Space representation of the simplex and hypercube, in other words, as the intersection of a set of linear inequalities and equalities defining half-spaces and hyperplanes. For now we can focus on the unconstrained cases for the standard $(n-1)$ -simplex, and the standard hypercube. 

In [1]:
function generate_simplex_polyhedron(n)
    coords = -I(n+1)
    non_neg_constraints = map(ax -> HalfSpace(coords[ax, :], 0.0), axes(coords, 1))
    simplex_constraint = HyperPlane(ones(n+1), 1.0)
    all_constraints = reduce(intersect, non_neg_constraints) ∩ simplex_constraint
    return polyhedron(all_constraints, CDDLib.Library())
end

function generate_hypercube_polyhedron(n)
    coords = I(n)
    upper_bound_constraints = map(ax -> HalfSpace(-coords[ax, :], 1.0), axes(coords, 1))
    lower_bound_constraints = map(ax -> HalfSpace(coords[ax, :], 1.0), axes(coords, 1))
    all_constraints = cat(upper_bound_constraints, lower_bound_constraints, dims=1)
    all_constraints = reduce(intersect, all_constraints)
    return polyhedron(all_constraints, CDDLib.Library())
end

get_vertices = (p) -> collect(points(p))

#7 (generic function with 1 method)

In [9]:
# H-space representation
simplex = generate_simplex_polyhedron(4)

# V-space representation
get_vertices(simplex)

5-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 1.0]
 [0.0, 0.0, 0.0, 1.0, 0.0]
 [0.0, 0.0, 1.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]

In [7]:
hypercube = generate_hypercube_polyhedron(4)
get_vertices(hypercube)

16-element Vector{Vector{Float64}}:
 [1.0, -1.0, -1.0, -1.0]
 [1.0, -1.0, -1.0, 1.0]
 [1.0, -1.0, 1.0, -1.0]
 [1.0, -1.0, 1.0, 1.0]
 [1.0, 1.0, 1.0, -1.0]
 [1.0, 1.0, 1.0, 1.0]
 [1.0, 1.0, -1.0, 1.0]
 [1.0, 1.0, -1.0, -1.0]
 [-1.0, 1.0, 1.0, 1.0]
 [-1.0, 1.0, 1.0, -1.0]
 [-1.0, 1.0, -1.0, 1.0]
 [-1.0, 1.0, -1.0, -1.0]
 [-1.0, -1.0, 1.0, 1.0]
 [-1.0, -1.0, 1.0, -1.0]
 [-1.0, -1.0, -1.0, 1.0]
 [-1.0, -1.0, -1.0, -1.0]

In [7]:
hrep(simplex)

H-representation CDDInequalityMatrix{Float64, Float64}:
1-element iterator of HyperPlane{Float64, Vector{Float64}}:
 HyperPlane([1.0, 1.0, 1.0], 1.0),
3-element iterator of HalfSpace{Float64, Vector{Float64}}:
 HalfSpace([-1.0, 0.0, 0.0], 0.0)
 HalfSpace([0.0, -1.0, 0.0], 0.0)
 HalfSpace([0.0, 0.0, -1.0], 0.0)

In [8]:
vrep(simplex)

V-representation CDDGeneratorMatrix{Float64, Float64}:
3-element iterator of Vector{Float64}:
 [0.0, 0.0, 1.0]
 [0.0, 1.0, 0.0]
 [1.0, 0.0, 0.0]

# Sample from Hypersphere
We need to sample from an $n$-dimensional sphere to get a random direction in which to travel for taking our sample.

In [ ]:
# Generate num_samples samples uniformly distributed over the surface of the unit n-dim hypersphere
function sample_random_direction(num_samples, n)
    v = randn(num_samples, n)
    return v ./ norm.(eachrow(v))
end

sample_random_direction (generic function with 1 method)

# Find Active Bounds by Computing Intersection
We need to be able to solve for the intersection of the ray from our current sampled point in the feasible region along the sampled direction vector with the nearest face of the bounding polyhedron.

In [19]:
# Define simplex constraints
A = [0 0 -1; 0 -1 0; -1 0 0; 1 1 1]
b = [0, 0, 0, 1]

# Define direction vector and current point
v = [1/3, 1/3, 1/3]
d = sample_random_direction(2, 3)[1,:]

size(A), size(v), size(b)

((4, 3), (3,), (4,))

In [20]:
# Provides bounds for t in a parameterized line v + td inside the polytope defined by Ax <= b
function get_bounds(A, b, v, d)
    unscaled_bounds = (b .- (A * v))[:, 1]
    scale_factors = (A * d)[:, 1]
    bounds = unscaled_bounds ./ scale_factors

    # We only care about bounds in the positive direction from our current point
    return minimum(bounds[scale_factors .> 0])
end

get_bounds (generic function with 1 method)

In [22]:
generate_simplex_polyhedron(3)

Polyhedron CDDLib.Polyhedron{Float64}:
1-element iterator of HyperPlane{Float64, Vector{Float64}}:
 HyperPlane([1.0, 1.0, 1.0], 1.0),
3-element iterator of HalfSpace{Float64, Vector{Float64}}:
 HalfSpace([-1.0, 0.0, 0.0], 0.0)
 HalfSpace([0.0, -1.0, 0.0], 0.0)
 HalfSpace([0.0, 0.0, -1.0], 0.0)

In [21]:
p = polyhedron(hrep(A, b, BitSet([4])), CDDLib.Library())

Polyhedron CDDLib.Polyhedron{Float64}:
1-element iterator of HyperPlane{Float64, Vector{Float64}}:
 HyperPlane([1.0, 1.0, 1.0], 1.0),
3-element iterator of HalfSpace{Float64, Vector{Float64}}:
 HalfSpace([0.0, 0.0, -1.0], 0.0)
 HalfSpace([0.0, -1.0, 0.0], 0.0)
 HalfSpace([-1.0, 0.0, 0.0], 0.0)

Polyhedron CDDLib.Polyhedron{Float64}:
2-element iterator of HyperPlane{Float64, Vector{Float64}}:
 HyperPlane([1.0, 1.0, 1.0], 1.0)
 HyperPlane([1.0, 1.0, 1.0], 1.0),
6-element iterator of HalfSpace{Float64, Vector{Float64}}:
 HalfSpace([0.0, 0.0, -1.0], 0.0)
 HalfSpace([0.0, -1.0, 0.0], 0.0)
 HalfSpace([-1.0, 0.0, 0.0], 0.0)
 HalfSpace([0.0, 0.0, -1.0], 0.0)
 HalfSpace([0.0, -1.0, 0.0], 0.0)
 HalfSpace([-1.0, 0.0, 0.0], 0.0)

# Hit and Run
What I want is a function that takes an H-Space representation of a convex polytope and repeatedly applies the hit-and-run MCMC algorithm to generate a set of random samples from the space.

In [ ]:
# For polytope Ax <= b,
# Run h&r n times inside the polytope starting from vector v
function hit_and_run(A, b, v, n)
    n_size = length(v)

    # Get n random directions
    rand_directions = sample_random_direction(n, n_size)
    
    samples = zeros(n, n_size)
    for i in 1:n
        d = rand_directions[i, :]
        t_max = get_bounds(A, b, v, d)
        t = rand() * t_max
        sample = v + t .* d
        samples[i, :] = sample
        v = sample
    end

    return samples
end 

hit_and_run (generic function with 1 method)

In [30]:
function rejection_sample(A, b)
    N, n = size(A)
    satisfies = false 
    x = ones(n)
    while satisfies == false
        x = rand(n)
        satisfies = sum((A * x) .<= b) == N
    end
    return x 
end

function hit_and_run(A, b, n)
    initial_sample = rejection_sample(A, b)
    return hit_and_run(A, b, initial_sample, n)
end

hit_and_run (generic function with 2 methods)

In [29]:
size(A, 1)

4

In [31]:
rejection_sample(A, b)

3-element Vector{Float64}:
 0.20350048161984513
 0.18175320023665054
 0.21367003128489115

In [23]:
hit_and_run(A, b, 10)

10×3 Matrix{Float64}:
  0.220943    0.497561   0.929175
  0.202802    0.2897     0.815535
 -1.0157      1.01191    1.02129
 -0.41007    -3.85039   -1.0201
 -2.81509    -4.22228   -0.463554
 -2.93231    -5.60407   -0.187843
 -0.0458197  -3.06527    0.611752
  1.26079    -1.2695    -0.516996
  0.917946   -0.844341   0.682753
  0.921709   -0.835595   0.680533

# Validation
At the very least, the arithmetic mean of the sample should be close to the centroid of the polytope. 

In [73]:
vertices = get_vertices(simplex)
centroid = sum(vertices) / length(vertices)

3-element Vector{Float64}:
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333

In [30]:
vertices

3-element Vector{Vector{Float64}}:
 [0.0, 0.0, 1.0]
 [0.0, 1.0, 0.0]
 [1.0, 0.0, 0.0]

In [27]:
samples = hit_and_run(A, b, v, 1000)

1000×3 Matrix{Float64}:
 0.333333    0.333333   0.333333
 0.333333    0.333333   0.333333
 0.333333    0.333333   0.333333
 0.302668    0.286526   0.362843
 0.306444    0.284013   0.362978
 0.419858    0.0468217  0.184226
 0.209967    0.226953   0.15231
 0.213046    0.225501   0.146308
 0.240534    0.163616   0.275005
 0.202703    0.179351   0.275501
 ⋮                      
 0.00442124  0.53526    0.417842
 0.153479    0.472082   0.222536
 0.0507364   0.391918   0.0829171
 0.165069    0.556581   0.124911
 0.117384    0.579771   0.143322
 0.115534    0.656849   0.193483
 0.259159    0.3105     0.0332886
 0.259088    0.310236   0.0328499
 0.370893    0.283849   0.0294364

In [40]:
sum(samples, dims=1) / size(samples, 1)

1×3 Matrix{Float64}:
 0.230485  0.225296  0.232473